In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/fisheye-camera-deep-learning/test.csv
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/train.json
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test/aa85791200d6.jpg
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test/cc9e335331a2.jpg
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test/d77f1d8c2855.jpg
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test/09ff87d7cc19.jpg
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test/649c3292ff4d.jpg
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test/5d2e2afa9c26.jpg
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test/ea3f0b381f7e.jpg
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test/c42e341ea2e5.jpg
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test/90d97c77632e.jpg
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test/ed234e03d6ce.jpg
/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test/e497

In [2]:
!pip -q install -U ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 21.0 MB/s eta 0:00:00


In [3]:
import os, json, random, shutil
from pathlib import Path
import numpy as np
import pandas as pd

# Paths Kaggle típicos
DATA_ROOT_1 = "/kaggle/input/fisheye-camera-deep-learning/"
DATA_ROOT_2 = "/kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye"
TRAIN_DIR = os.path.join(DATA_ROOT_2, "train")
TEST_DIR  = os.path.join(DATA_ROOT_2, "test")
TRAIN_JSON = os.path.join(DATA_ROOT_2, "train.json")
TEST_CSV   = os.path.join(DATA_ROOT_1, "test.csv")

print("TRAIN_DIR:", TRAIN_DIR)
print("TEST_DIR:", TEST_DIR)
print("TRAIN_JSON:", TRAIN_JSON)
print("TEST_CSV:", TEST_CSV)

OUT_ROOT = "/kaggle/working/yolo_obb_kaggle"
os.makedirs(OUT_ROOT, exist_ok=True)

TRAIN_DIR: /kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/train
TEST_DIR: /kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/test
TRAIN_JSON: /kaggle/input/fisheye-camera-deep-learning/Fisheye/Fisheye/train.json
TEST_CSV: /kaggle/input/fisheye-camera-deep-learning/test.csv


In [4]:
import math

def rbox_to_poly(cx, cy, w, h, angle_deg):
    """
    Convierte caja rotada (cx,cy,w,h,angle_deg) a 4 puntos (x1,y1,...,x4,y4)
    cx,cy en pixeles (origen top-left)
    angle_deg: grados (convención del reto: sentido horario desde eje vertical)
    """
    # Convertimos a radianes
    ang = math.radians(angle_deg)

    # Si el ángulo está medido desde eje vertical, una forma simple es rotar sistema:
    # Tomamos base rectángulo alineado y rotamos en el plano con matriz.
    # Este ajuste depende de convención exacta; para el reto suele funcionar así:
    # - interpretamos ang como rotación horaria desde eje vertical => equivalente a
    #   rotación anti-horaria desde eje x: theta = 90 - ang (en grados)
    theta = math.radians(90 - angle_deg)

    cos_t = math.cos(theta)
    sin_t = math.sin(theta)

    # half sizes
    dw = w / 2.0
    dh = h / 2.0

    # corners in local coords (x right, y down)
    corners = np.array([
        [-dw, -dh],
        [ dw, -dh],
        [ dw,  dh],
        [-dw,  dh],
    ], dtype=np.float32)

    # rotation matrix
    R = np.array([[cos_t, -sin_t],
                  [sin_t,  cos_t]], dtype=np.float32)

    pts = corners @ R.T
    pts[:, 0] += cx
    pts[:, 1] += cy

    return pts.reshape(-1)  # 8 numbers

In [5]:
def make_dir(p):
    os.makedirs(p, exist_ok=True)
    return p

IMG_OUT_TRAIN = make_dir(os.path.join(OUT_ROOT, "images", "train"))
LBL_OUT_TRAIN = make_dir(os.path.join(OUT_ROOT, "labels", "train"))
IMG_OUT_VAL   = make_dir(os.path.join(OUT_ROOT, "images", "val"))
LBL_OUT_VAL   = make_dir(os.path.join(OUT_ROOT, "labels", "val"))

with open(TRAIN_JSON, "r") as f:
    coco = json.load(f)

images = coco["images"]
annotations = coco["annotations"]

# agrupar anns por image_id
ann_by_image = {}
for a in annotations:
    ann_by_image.setdefault(a["image_id"], []).append(a)

# split train/val
random.seed(42)
img_items = images.copy()
random.shuffle(img_items)

val_ratio = 0.2
n_val = int(len(img_items) * val_ratio)
val_items = img_items[:n_val]
train_items = img_items[n_val:]

print("Total images:", len(img_items), "train:", len(train_items), "val:", len(val_items))

def link_or_copy(src, dst):
    try:
        if os.path.exists(dst):
            return
        os.symlink(src, dst)
    except Exception:
        shutil.copy2(src, dst)

def write_yolo_obb_label(img_item, out_txt_path, class_id=0):
    """
    YOLO-OBB label format (Ultralytics):
    class x1 y1 x2 y2 x3 y3 x4 y4  (normalized 0-1)
    """
    W = img_item["width"]
    H = img_item["height"]
    image_id = img_item["image_id"]
    anns = ann_by_image.get(image_id, [])
    lines = []

    for a in anns:
        cx, cy, w, h, ang = a["bbox"]

        # ✅ CONVERSION CLAVE: origen en centro -> origen top-left
        cx = cx + W/2.0
        cy = cy + H/2.0

        poly = rbox_to_poly(cx, cy, w, h, ang)  # 8 vals en pixeles

        # normalizar
        poly_norm = poly.copy()
        poly_norm[0::2] /= W
        poly_norm[1::2] /= H

        # clamp por seguridad (a veces hay bordes)
        poly_norm = np.clip(poly_norm, 0.0, 1.0)

        line = " ".join([str(class_id)] + [f"{v:.6f}" for v in poly_norm.tolist()])
        lines.append(line)

    with open(out_txt_path, "w") as f:
        f.write("\n".join(lines))

def convert_split(items, img_out_dir, lbl_out_dir):
    for im in items:
        fname = im["file_name"]
        src_img = os.path.join(TRAIN_DIR, fname)
        dst_img = os.path.join(img_out_dir, fname)
        link_or_copy(src_img, dst_img)

        base = Path(fname).stem
        dst_lbl = os.path.join(lbl_out_dir, f"{base}.txt")
        write_yolo_obb_label(im, dst_lbl, class_id=0)

convert_split(train_items, IMG_OUT_TRAIN, LBL_OUT_TRAIN)
convert_split(val_items, IMG_OUT_VAL,   LBL_OUT_VAL)

print("Ejemplo label:", sorted(Path(LBL_OUT_TRAIN).glob("*.txt"))[:1])

Total images: 20656 train: 16525 val: 4131
Ejemplo label: [PosixPath('/kaggle/working/yolo_obb_kaggle/labels/train/0005c6244a05.txt')]


In [6]:
data_yaml = f"""
path: {OUT_ROOT}
train: images/train
val: images/val

names:
  0: person
"""

yaml_path = os.path.join(OUT_ROOT, "data.yaml")
with open(yaml_path, "w") as f:
    f.write(data_yaml)

print("YAML:", yaml_path)
print(data_yaml)

YAML: /kaggle/working/yolo_obb_kaggle/data.yaml

path: /kaggle/working/yolo_obb_kaggle
train: images/train
val: images/val

names:
  0: person



In [7]:
### DONDE ESTA EL CENTRO SEGUN JSON? ###
import json, numpy as np

with open(TRAIN_JSON, "r") as f:
    coco = json.load(f)

# toma 5000 bboxes para mirar rangos
b = np.array([a["bbox"] for a in coco["annotations"][:5000]])  # cx,cy,w,h,ang
cx, cy = b[:,0], b[:,1]

print("cx min/max:", cx.min(), cx.max())
print("cy min/max:", cy.min(), cy.max())

# mira una imagen para saber W,H típico
im0 = coco["images"][0]
print("Example W,H:", im0["width"], im0["height"])

cx min/max: 108.29290000000003 929.8273999999999
cy min/max: 91.424 773.7099
Example W,H: 1080 1080


In [8]:
import torch, os
print("cuda available:", torch.cuda.is_available())
print("device count:", torch.cuda.device_count())
print("CUDA_VISIBLE_DEVICES:", os.environ.get("CUDA_VISIBLE_DEVICES"))

cuda available: True
device count: 2
CUDA_VISIBLE_DEVICES: None


In [9]:
### TRAINING ###
from ultralytics import YOLO

model = YOLO("yolov8s-obb.pt")  # prueba también yolov8n-obb.pt si quieres más rápido

results = model.train(
    data=yaml_path,
    epochs=60, # SUGERENCIA: probar con valores entre 50-100
    imgsz=1024, # SUGERENCIA: imgsz 1024
    batch=8,
    device="0,1",
    workers=2,
    project="/kaggle/working",
    name="yolo_obb_fisheye",
    exist_ok=True
)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.4.9 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 14913MiB)
                                                     CUDA:1 (Tesla T4, 14913MiB)
engine/trainer: agnostic_nms=False, amp=True, angle=1.0, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/kaggle/working/yolo_obb_kaggle/data.yaml, degrees=0.0, deterministic=True, device=0,1, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, end2end=None, epochs=60, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript,

In [10]:
### TESTING: LEER CSV ###
test_df = pd.read_csv(TEST_CSV)
test_ids = test_df["Id"].tolist()
print("Test images:", len(test_ids))
print("Ejemplo:", test_ids[:3])

Test images: 4848
Ejemplo: ['02a06e163d03', 'a77bf7ff0b33', '1cd8913ec737']


In [11]:
### TESTING: FUNCION POLYGON ###
import cv2

def poly_to_rbox(poly_xy):
    """
    poly_xy: array shape (4,2) en pixeles
    return: cx, cy, w, h, angle_deg (en grados)
    """
    rect = cv2.minAreaRect(poly_xy.astype(np.float32))
    (cx, cy), (w, h), angle = rect

    # OpenCV devuelve angle en convención propia: [-90, 0)
    # Ajustamos para mantener w>=h como convención estable
    if w < h:
        w, h = h, w
        angle = angle + 90

    # Ahora angle es respecto al eje x (horizontal) CCW.
    # El reto dice: "horario desde eje vertical".
    # Convertimos: angle_from_vertical_clockwise = 90 - angle_x_ccw
    angle_deg = 90 - angle

    # Normalizamos a [-180,180] opcional
    while angle_deg > 180:
        angle_deg -= 360
    while angle_deg < -180:
        angle_deg += 360

    return cx, cy, w, h, angle_deg

In [12]:
### TESTING: INFERENCIA Y ARMADO STRINGS ###

from ultralytics import YOLO

# Cargar mejor modelo entrenado
best_path = "/kaggle/working/yolo_obb_fisheye/weights/best.pt"
model = YOLO(best_path)

pred_strings = []

for img_id in test_ids:
    img_path = os.path.join(TEST_DIR, f"{img_id}.jpg")

    out = model.predict(
        source=img_path,
        imgsz=1024,
        conf=0.10,
        iou=0.5,
        verbose=False
    )[0]

    H, W = out.orig_shape

    if out.obb is None or len(out.obb) == 0:
        pred_strings.append("0")
        continue

    det_strs = []
    polys = out.obb.xyxyxyxy.cpu().numpy()
    scores = out.obb.conf.cpu().numpy()

    for poly, score in zip(polys, scores):
        poly_pts = poly.reshape(4, 2)

        cx_px, cy_px, w_px, h_px, ang_deg = poly_to_rbox(poly_pts)

        # OJO: revisa esto (te explico abajo)
        cx = cx_px
        cy = cy_px

        det_strs.append(
            f"{score:.3f} {cx:.2f} {cy:.2f} {w_px:.2f} {h_px:.2f} {ang_deg:.2f}"
        )

    pred_strings.append("; ".join(det_strs))

In [13]:
### SI ALGO FALLA... ###
print(out)
print(dir(out))
print(out.obb)

ultralytics.engine.results.Results object with attributes:

boxes: None
keypoints: None
masks: None
names: {0: 'person'}
obb: ultralytics.engine.results.OBB object
orig_img: array([[[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       ...,

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]],

       [[0, 0, 0],
        [0, 0, 0],
        [0, 0, 0],
        ...,
        [0, 0, 0],
        [0, 0, 0],
        [0, 0, 0]]], dtype=uint8)
orig_shape: (2048, 2048)
p

In [14]:
### CREAR SUBMISSION ###
sub = pd.DataFrame({"Id": test_ids, "Predicted": pred_strings})
sub_path = "/kaggle/working/submission.csv"
sub.to_csv(sub_path, index=False)
print("Saved:", sub_path)
sub.head()

Saved: /kaggle/working/submission.csv


,Id,Predicted
0,02a06e163d03,0.968 988.17 751.80 99.40 58.23 21.57; 0.964 9...
1,a77bf7ff0b33,0.955 983.16 770.48 118.45 60.74 -6.13; 0.868 ...
2,1cd8913ec737,0.968 987.85 751.82 99.37 58.03 21.49; 0.961 1...
3,62e2519d2391,0.965 988.89 750.04 103.50 59.97 15.95; 0.939 ...
4,c1639da239ec,0.961 988.10 751.69 100.06 59.68 20.57; 0.923 ...


In [15]:
import re

def ok(pred):
    pred = str(pred).strip()
    if pred == "0":
        return True
    dets = [d.strip() for d in pred.split(";")]
    for d in dets:
        nums = re.split(r"\s+", d)
        if len(nums) != 6:
            return False
    return True

print(sub["Predicted"].apply(ok).all())

True


In [16]:
"""
OTRAS MEJORAS SUGERIDAS:
- TTA (test-time augmentation): Ultralytics tiene augment=True en predict (más lento pero a veces sube AP).
- Validación: mira el mAP en val y ajusta conf/iou.
"""

'\nOTRAS MEJORAS SUGERIDAS:\n- TTA (test-time augmentation): Ultralytics tiene augment=True en predict (más lento pero a veces sube AP).\n- Validación: mira el mAP en val y ajusta conf/iou.\n'